# Overview
This notebook serves as a brief example of how to make changes to the model through script. There are four functions we will be using:
1. get_param function: returns the parameter's value, given a specific context (node, year, technology, and sub-parameter)
2. set_param function: sets a parameter's values, given a specific context (node, year, technology, and sub-parameter)
3. set_param_wildcard function: sets a parameter's value, for all contexts (node, year, technology, and sub-parameter) where the node satisfies/matches a specified pattern
4. set_param_log: saves the changes made to the model into a CSV file

# Build the model
We need to first have the model ready before using the function. All steps of building the model are the same as indicated in the Quickstart notebook. We do not run the model yet as we need to make the changes beforehand.

In [ ]:
import pyCIMS
import pprint as pp

# description file
model_description_file = '../../model_descriptions/pyCIMS_model_description_ALL_value.xlsb'

# model validator
model_validator = pyCIMS.ModelValidator(model_description_file, 
                                        sheet_map={'model': 'Lists',
                                                   'default_tech': 'Technology_Node templates'},
                                        node_col='Node')
model_validator.validate(verbose=True, raise_warnings=False)

# Model Reader
model_reader = pyCIMS.ModelReader(infile=model_description_file,
                                  sheet_map={'model': 'Lists', 
                                             'default_tech': 'Technology_Node templates'},
                                  node_col='Node')

# Model
model = pyCIMS.Model(model_reader)

This function is defined mainly for formatting purposes for this notebook:

In [ ]:
def print_formatted(prev_value, curr_value, param, node, year=None, tech=None, sub_param=None):
    dash = '-' * 50
    print('Parameter: ' + param)
    print('Node: ' + node)
    if year:
        print('Year: ' + year)
    if tech:
        print('Tech: ' + tech)
    if sub_param:
        print('Sub-parameter: ' + sub_param)
    print(dash)
    print("Previous value: " + str(prev_value))
    print("New value: " + str(curr_value))
    print('\n')

# 1. The get_param function

<div class="alert alert-block alert-success">
<b></b> The function has 5 arguments : param, node, year, tech, sub_param.
</div>

This function is under the `model` class and has 2 required arguments:
* param : the name of the parameter whose value is to be set
* node : the name of the node whose parameter you are interested in

There are also 3 optional arguments:
* year : The year which you are interested in. The default value set to None.
* tech : The name of the tecnology you are interested in. The default value set to None.
* sub_param : The sub-parameter you are interested in (used to specify a nested key). The default value set to None.

### Example Usages

1. Get the Market share value in year "2000" for the Incandescent technology at the node pyCIMS.Canada.Alberta.Residential.Buildings.Floorspace.Lighting

In [ ]:
#Set parameters
param = 'Market share'
node = 'pyCIMS.Canada.Alberta.Residential.Buildings.Floorspace.Lighting'
year = '2000'
tech = 'Incandescent'

model.get_param(param=param, node=node, year=year, tech=tech)

2. Get the Service requested value in the year "2010" for the sub-parameter Old Truck at the node pyCIMS.Canada.Alberta.Transportation Personal.Passenger Vehicles.Existing

In [ ]:
#Set parameters
param = 'Service requested'
node = 'pyCIMS.Canada.Alberta.Transportation Personal.Passenger Vehicles.Existing'
year = '2010'
sub_param = 'Old Truck'

model.get_param(param=param, node=node, year=year, sub_param=sub_param)

# 2. The set_param function

<div class="alert alert-block alert-success">
<b></b> The function has 7 arguments : val, param, node, year, tech, sub_param, save.
</div>

This function is under the `model` class and has 3 required arguments:
* val : the new value to be set
* param : the name of the parameter whose value is to be set
* node : the name of the node whose parameter you are interested in

There are also 4 optional arguments:
* year : The year which you are interested in. The default value set to None.
* tech : The name of the tecnology you are interested in. The default value set to None.
* sub_param : The sub-parameter you are interested in (used to specify a nested key). The default value set to None.
* save : A boolean value that specifies whether you want the change to be recorded in a change log CSV file. The default value set to True.

Note: In the case that you want to set a range of values for a range of years, you can pass a list of new values into `val` and a list of corresponding years into `year`. These two lists would need to be of the same length.

### Example Usages

1. At node pyCIMS.Canada.Alberta.Residential.Buildings.Floorspace.Lighting update the Market share in year "2000" for the Incandescent & CFL technologies to 0.8 and 0.2, respectively.

In [ ]:
#Set parameters
param = 'Market share'
node = 'pyCIMS.Canada.Alberta.Residential.Buildings.Floorspace.Lighting'
year = '2000'
tech = 'Incandescent'
val = 0.8

#Get the original value for printing purposes
prev_value = model.get_param(param=param, node=node, year=year, tech=tech)

#Change the value
model.set_param(val=val, param=param, node=node, year=year, tech=tech)

#Print the results
curr_value = model.get_param(param=param, node=node, year=year, tech=tech)
print_formatted(prev_value=prev_value, curr_value=curr_value, param=param, node=node, year=year, tech=tech)

In [ ]:
#Set parameters
param = 'Market share'
node = 'pyCIMS.Canada.Alberta.Residential.Buildings.Floorspace.Lighting'
year = '2000'
tech = 'CFL'
val = 0.2

#Get the original value for printing purposes
prev_value = model.get_param(param=param, node=node, year=year, tech=tech)

#Change the value
model.set_param(val=val, param=param, node=node, year=year, tech=tech)

#Print the results
curr_value = model.get_param(param=param, node=node, year=year, tech=tech)
print_formatted(prev_value=prev_value, curr_value=curr_value, param=param, node=node, year=year, tech=tech)

2. At node pyCIMS.Canada.Alberta.Transportation Personal.Passenger Vehicles.Existing update the Service requested value in all years for the Recent Car and Old Truck technologies to 0.7 and 0.3, respectively.

In [ ]:
#Set parameters
param = 'Service requested'
node = 'pyCIMS.Canada.Alberta.Transportation Personal.Passenger Vehicles.Existing'
years = model.years
sub_param = 'Recent Car'
new_values = [0.7]*len(years)

#Get the original value(s) for printing purposes
prev_vals = []
for year in years:
    prev_vals.append(model.get_param(param=param, node=node, year=year, sub_param=sub_param))

#Value(s) are changed
model.set_param(val=new_values, param=param, node=node, year=years, sub_param=sub_param)

#Print the results
for i, year in enumerate(years):
    curr_value = model.get_param(param=param, node=node, year=year, sub_param=sub_param)
    print_formatted(prev_value=prev_vals[i], curr_value=curr_value, param=param, node=node, year=year, sub_param=sub_param)

In [ ]:
#Set parameters
param = 'Service requested'
node = 'pyCIMS.Canada.Alberta.Transportation Personal.Passenger Vehicles.Existing'
years = model.years
sub_param = 'Old Truck'
new_values = [0.3]*len(years)

#Get the original value(s) for printing purposes
prev_vals = []
for year in years:
    prev_vals.append(model.get_param(param=param, node=node, year=year, sub_param=sub_param))

#Value(s) are changed
model.set_param(val=new_values, param=param, node=node, year=years, sub_param=sub_param)

#Print the results
for i, year in enumerate(years):
    curr_value = model.get_param(param=param, node=node, year=year, sub_param=sub_param)
    print_formatted(prev_value=prev_vals[i], curr_value=curr_value, param=param, node=node, year=year, sub_param=sub_param)

3. Set Price Multiplier values in all years for all sector nodes (competition type == Sector or Sector No Tech) to 3.0

Note: We put a try and except block since the nodes we are searching may not have the Price Multiplier parameter and we would like to skip these nodes

In [ ]:
#Set parameters
param = 'Price Multiplier'
new_value = 3.0

for node in model.graph.nodes:
    #Check if the node's competition type is either 'sector' or 'sector no tech '
    if model.get_param('competition type', node) in ['sector', 'sector no tech']:
        for year in model.years:
            try:
                #Retrieve all possible sub-parameters under Price Multiplier for the specified node and year
                sub_params = list(model.get_param(param=param, node=node, year=year).keys())
            except: 
                continue
            for sub_param in sub_params:
                #Get the original value(s) for printing purposes
                prev_val = model.get_param(param=param, node=node, year=year, sub_param=sub_param)

                #Value(s) are changed
                model.set_param(val=new_value, param=param, node=node, year=year, sub_param=sub_param)

                #Print the results
                curr_val = model.get_param(param=param, node=node, year=year, sub_param=sub_param)
                print_formatted(prev_value=prev_val, curr_value=curr_val, param=param, node=node, year=year, sub_param=sub_param)
            

# 3. The set_param_wildcard function

<div class="alert alert-block alert-success">
<b></b> The function has 7 arguments : val, param, node_regex, year, tech, sub_param, save.
</div>

This function is under the `model` class and has 3 required arguments:
* val : the new value to be set
* param : the name of the parameter whose value is to be set
* node_regex : the regex pattern of the node (branch format) whose parameter you are interested in matching.

There are also 4 optional arguments:
* year : The year which you are interested in. The default value set to None.
* tech : The name of the tecnology you are interested in. The default value set to None.
* sub_param : The sub-parameter you are interested in (used to specify a nested key). The default value set to None.
* save : A boolean value that specifies whether you want the change to be recorded in a change log CSV file. The default value set to True.

Note: In the case that you want to set a range of values for a range of years, you can pass a list of new values into `val` and a list of corresponding years into `year`. These two lists would need to be of the same length.

### Regex
Regex is short for regular expression and is a sequence of characters that specifies a search pattern. There are many resources online (https://www.rexegg.com/regex-quickstart.html) to learn more about writing Regex expressions but here are some basic ideas we will be using in the following examples:
1. Characters:
    1. `.` represents any character (except line break)
    2. `\` escapes a special character. For example, `.` means any character but `\.` means a period. 
2. Quantifiers:
    1. `*` means zero or more times
    2. `+` means one or more times
3. Anchors and Boundaries:
    1. `^` means the start of a string
    2. `$` means the end of the string
    
You can use https://regex101.com/ to check whether your regex expression is working as expected

### Example Regex Expressions

#### Example 1
Match any string that ends with 'Pumping.Precision.Small' (e.g. pyCIMS.Canada.Alberta.Coal Mining.Pumping.Precision.Small, pyCIMS.Canada.Alberta.Pulp  Paper.Pumping.Precision.Small etc)

Let's look at how to build the corresponding Regex expression. 

We want to be able to search for all node names that end with the string 'Pumping.Precision.Small'. In Regex, since `.` is a special character representing any character, we will write the string as `'Pumping\.Precision\.Small'`, where `\.` represents a period. 

We want to look for node names that end with this string so we will add `.*` to the front of the string and `$` to the end. `.*` specifies that we can have zero or more of any character before the string and `$` specifies that there should be nothing after this string.

The final resulting Regex expression is `'.*Pumping\.Precision\.Small$'`

#### Example 2
Match any string that starts with 'pyCIMS.Canada.Alberta.Residential.Buildings.Floorspace.' (e.g. pyCIMS.Canada.Alberta.Residential.Buildings.Floorspace.Solar Electricity, pyCIMS.Canada.Alberta.Residential.Buildings.Floorspace.Space Conditioning.Apartments etc)

Let's look at how to build the corresponding Regex expression. 

We want to be able to search for all node names that start with the string 'pyCIMS.Canada.Alberta.Residential.Buildings.Floorspace.'. As before, we will begin with the string as `'pyCIMS\.Canada\.Alberta\.Residential\.Buildings\.Floorspace\.'` where `\.` represents a period. 

We want to look for nodes names that start with this string so we will add `^` to the start and `.*` to the end of the string. `^` specifies that there should be nothing before this string and `.*` specifies that we can have zero or more of any character after the string.

The final resulting Regex expression is `'^pyCIMS\.Canada\.Alberta\.Residential\.Buildings\.Floorspace\..*'`

#### Example 3
Match any string that has '.Residential.Buildings.Floorspace.' anywhere in the string except at the very front or very end of the string (e.g. pyCIMS.Canada.Alberta.Residential.Buildings.Floorspace.Lighting, pyCIMS.Canada.Alberta.Residential.Buildings.Floorspace.Space Conditioning.Single Family Attached etc)

Let's look at how to build the corresponding Regex expression. 

We want to be able to search for all node names that contain the string '.Residential.Buildings.Floorspace.'. As before, we will begin with the string as `'\.Residential\.Buildings\.Floorspace\.'` where `\.` represents a period. 

We want to look for nodes names that contains this string. We will add `.+` to the front and back of the string where `.+` represents one or more of any character.

The final resulting Regex expression is `'.+\.Residential\.Buildings\.Floorspace\..+'`

### Example set_param_wildcard Usages

1. For all nodes where their branches match the form *Pumping.Precision.Small (e.g. pyCIMS.Canada.Alberta.Coal Mining.Pumping.Precision.Small, pyCIMS.Canada.Alberta.Pulp  Paper.Pumping.Precision.Small, pyCIMS.Canada.Alberta.Chemical Products.Pumping.Precision.Small, etc) change the value of Heterogeneity (across all years) to 0.6.

Note: We put a try and except block around the get_param functions since the nodes we are searching may not have a Heterogeneity parameter and we would like to skip these nodes when printing the results.

In [ ]:
#Import the Python Regex library
import re

#Set parameters
param = 'Heterogeneity'
years = model.years
vals = [0.6]*len(years)
node_regex = '.*Pumping\.Precision\.Small$'

#Get the original value(s) just for printing purposes
prev_vals = []
for node in model.graph.nodes:
    if re.search(node_regex, node) != None:
        for year in model.years:
            try:
                prev_vals.append(model.get_param(param=param, node=node, year=year))
            except:
                continue

# Value(s) are changed
model.set_param_wildcard(val=vals, param=param, node_regex=node_regex, year=years)

#Print the results
i = 0
for node in model.graph.nodes:
    if re.search(node_regex, node) != None:
        for year in model.years:
            try:
                curr_value = model.get_param(param=param, node=node, year=year)
            except:
                continue
            print_formatted(prev_value=prev_vals[i], curr_value=curr_value, param=param, node=node, year=year)
            i += 1

2. For all nodes where their branches match the form pyCIMS.Canada.Alberta.Residential.Buildings.Floorspace.* (e.g. pyCIMS.Canada.Alberta.Residential.Buildings.Floorspace.Solar Electricity, pyCIMS.Canada.Alberta.Residential.Buildings.Floorspace.Space Conditioning.Apartments etc) change the Service Requested for Lighting in 2000 to 1.3

Note: We put a try and except block around the get_param functions since the nodes we are searching may not have a Heterogeneity parameter and we would like to skip these nodes when printing the results.

In [ ]:
#Import the Python Regex library
import re

#Set parameters
param = 'Heterogeneity'
year = '2000'
vals = 1.3
node_regex = '^pyCIMS\.Canada\.Alberta\.Residential\.Buildings\.Floorspace\..*'

#Get the original value(s) just for printing purposes
prev_vals = []
for node in model.graph.nodes:
    if re.search(node_regex, node) != None:
        try:
            prev_vals.append(model.get_param(param=param, node=node, year=year))
        except:
            continue

#Value(s) are changed
model.set_param_wildcard(val=vals, param=param, node_regex=node_regex, year=year)

#Print the results
i = 0
for node in model.graph.nodes:
    if re.search(node_regex, node) != None:
        try:
            curr_value = model.get_param(param=param, node=node, year=year)
        except:
            continue
        print_formatted(prev_value=prev_vals[i], curr_value=curr_value, param=param, node=node, year=year)
        i += 1

3. For all nodes where their branches match the form \*.Residential.Buildings.Floorspace.* (e.g. pyCIMS.Canada.Alberta.Residential.Buildings.Floorspace.Lighting, pyCIMS.Canada.Alberta.Residential.Buildings.Floorspace.Space Conditioning.Single Family Attached etc) change the Price Multiplier for Gasoline in '2010' to 1.2

Note: We put a try and except block around the get_param functions since the nodes we are searching may not have a Heterogeneity parameter and we would like to skip these nodes when printing the results.

In [ ]:
#Import the Python Regex library
import re

#Set parameters
param = 'Heterogeneity'
year = '2010'
val = 1.2
node_regex = '.+\.Residential\.Buildings\.Floorspace\..+'

#Get the original value(s) just for printing purposes
prev_vals = []
for node in model.graph.nodes:
    if re.search(node_regex, node) != None:
        try:
            prev_vals.append(model.get_param(param=param, node=node, year=year))
        except Exception as e:
            continue

#Value(s) are changed
model.set_param_wildcard(val=val, param=param, node_regex=node_regex, year=year)

#Print the results
i = 0
for node in model.graph.nodes:
    if re.search(node_regex, node) != None:
        try:
            curr_value = model.get_param(param=param, node=node, year=year)
            print_formatted(prev_value=prev_vals[i], curr_value=curr_value, param=param, node=node, year=year)
            i += 1
        except:
            continue

# Run the Model
We can now run the model training after all the changes have been applied.

In [ ]:
# run the model 
model.run(max_iterations=5, show_warnings=False)

# 4. The set_param_log function

<div class="alert alert-block alert-success">
<b></b> The function has 1 arguments : output_file.
</div>

This function is under the `model` class and only has 1 optional argument:
* output_file : the output file location where the change history CSV will be saved. If this is left blank, the file will be outputed at the current location with the name of the original model description and a timestamp in the filename.

### Example Usages

In [ ]:
#file is saved to ./change_log.csv
model.set_param_log(output_file='./change_log.csv') 

In [ ]:
#default saved to change_log_<model_description_file>_<timestamp>.csv
#model.set_param_log() 

# Saving and loading model
In order to save the model for later use, you can use the `save_model` function and `load_model` functions.

The `save_model` function is under the `model` class and has 2 optional arguments:
* model_file : The model file location where the model file will be saved. If this is left blank, the model will be saved at the current location with the name of the original model description and a timestamp in the filename.
* save_changes : a boolean value that specifies whether the changes will be saved to a CSV file with a similar filename as the model_file

In [ ]:
#file is saved to ./model_file_test.pkl and log of changes were not written to CSV 
model.save_model(model_file='model_file_test.pkl', save_changes=False) 

In [ ]:
#default saved to model_<model_description_file>_<timestamp>.pkl and log of changes were written to CSV
# model.save_model() 

The `load_model` function is under the `pyCIMS` class and has 1 required arguments:
* model_file : The model file location where the model file is saved.

In [ ]:
model = pyCIMS.load_model(model_file='model_file_test.pkl')